# Performance showcase of added "to_sql" functionality in mlinspect

Here the performance of the proposed inspection using sql will be compared to the original one in pandas. Part of
the "healthcare" and "compas" pipeline will be used.

## Required packages:
See: requirements/requirements.txt and requirements/requirements.dev.txt

## Some parameters you might want to set:

In [1]:
import os
import sys
import time

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from inspect import cleandoc
from mlinspect.utils import get_project_root
from mlinspect import PipelineInspector, OperatorType
from mlinspect.inspections import HistogramForColumns, RowLineage, MaterializeFirstOutputRows
from mlinspect.checks import NoBiasIntroducedFor, NoIllegalFeatures
from demo.feature_overview.no_missing_embeddings import NoMissingEmbeddings
from example_pipelines.healthcare import custom_monkeypatching
from mlinspect.to_sql.dbms_connectors.postgresql_connector import PostgresqlConnector

from mlinspect.to_sql.dbms_connectors.umbra_connector import UmbraConnector


# DBMS related:
UMBRA_USER = "postgres"
UMBRA_PW = ""
UMBRA_DB = ""
UMBRA_PORT = 5433
UMBRA_HOST = "/tmp/"

POSTGRES_USER = "luca"
POSTGRES_PW = "password"
POSTGRES_DB = "healthcare_benchmark"
POSTGRES_PORT = 5432
POSTGRES_HOST = "localhost"

pipe = cleandoc("""
    import warnings
    import os
    import pandas as pd
    from sklearn.pipeline import Pipeline
    from mlinspect.utils import get_project_root

    taxi = pd.read_csv(
        os.path.join( str(get_project_root()), "example_pipelines", "taxi", "yellow_tripdata_202101_head.csv"),
        na_values='?')

    taxi = taxi[(taxi['passenger_count']>=1)]
""")


## Benchmark setup:

In [2]:
def run_inspection(code, bias, to_sql, dbms_connector=None, mode=None, materialize=None):
    from PIL import Image
    import matplotlib.pyplot as plt
    from mlinspect.visualisation import save_fig_to_path

    inspector_result = PipelineInspector \
        .on_pipeline_from_string(code) \
        .add_custom_monkey_patching_module(custom_monkeypatching) \
        .add_check(NoBiasIntroducedFor(bias))

    if to_sql:
        inspector_result = inspector_result.execute_in_sql(dbms_connector=dbms_connector, mode=mode,
                                                           materialize=materialize)
    else:
        inspector_result = inspector_result.execute()

    check_results = inspector_result.check_to_check_results
    no_bias_check_result = check_results[NoBiasIntroducedFor(bias)]

    distribution_changes_overview_df = NoBiasIntroducedFor.get_distribution_changes_overview_as_df(
        no_bias_check_result)
    result = ""
    result += distribution_changes_overview_df.to_markdown()

    for i in list(no_bias_check_result.bias_distribution_change.items()):
        _, join_distribution_changes = i
        for column, distribution_change in join_distribution_changes.items():
            result += "\n"
            result += f"\033[1m Column '{column}'\033[0m"
            result += distribution_change.before_and_after_df.to_markdown()

    return result

## Benchmark of default inspection using CTEs:

In [4]:
#dbms_connector_u = UmbraConnector(dbname=UMBRA_DB, user=UMBRA_USER, password=UMBRA_PW,
#                                  port=UMBRA_PORT, host=UMBRA_HOST, add_mlinspect_serial=False)

dbms_connector_p = PostgresqlConnector(dbname=POSTGRES_DB, user=POSTGRES_USER, password=POSTGRES_PW,
                                       port=POSTGRES_PORT, host=POSTGRES_HOST)

def run_for_all(code, bias):
    t0 = time.time()
    #run_inspection(code=code, bias=bias, to_sql=False)
    t1 = time.time()
    print("\nOriginal: " + str(t1 - t0))

    t0 = time.time()
    run_inspection(code=code, bias=bias, to_sql=True, dbms_connector=dbms_connector_p, mode="VIEW",
                   materialize=None)
    t1 = time.time()
    print("\nPostgreSQL View: " + str(t1 - t0))
    
    t0 = time.time()
    run_inspection(code=code, bias=bias, to_sql=True, dbms_connector=dbms_connector_p, mode="VIEW",
                   materialize=True)
    t1 = time.time()
    print("\nPostgreSQL Materialized View: " + str(t1 - t0))

#   t0 = time.time()
#    run_inspection(code=code, bias=bias, to_sql=True, dbms_connector=dbms_connector_u, mode="VIEW",
#                   materialize=None)
#    t1 = time.time()
#    print("\nUmbra View: " + str(t1 - t0))
    
    t0 = time.time()
    run_inspection(code=code, bias=bias, to_sql=True, dbms_connector=dbms_connector_p, mode="CTE",
                   materialize=None)
    t1 = time.time()
    print("\nPostgreSQL CTE: " + str(t1 - t0))

#    t0 = time.time()
#    run_inspection(code=code, bias=bias, to_sql=True, dbms_connector=dbms_connector_u, mode="CTE",
#                   materialize=None)
#    t1 = time.time()
#    print("\nUmbra CTE: " + str(t1 - t0))


## End-to-End example of the preprocessing-pipeline inspection + model training:

Slightly different inspections results are expected because of the random split. Still, the resulting model accuracy should
be similar.

In [5]:
run_for_all(pipe, ['passenger_count'])


Original: 0.0
from mlinspect.instrumentation._pipeline_executor import set_code_reference_call, set_code_reference_subscript, monkey_patch, undo_monkey_patch
monkey_patch()
import warnings
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from mlinspect.utils import get_project_root
taxi = pd.read_csv(os.path.join(str(get_project_root(**
    set_code_reference_call(8, 22, 8, 40)), **set_code_reference_call(8, 18,
    8, 41)), 'example_pipelines', 'taxi', 'yellow_tripdata_202101_head.csv',
    **set_code_reference_call(8, 4, 8, 106)), **set_code_reference_call(7, 
    7, 9, 18, na_values='?'))
taxi = taxi[set_code_reference_subscript(11, 7, 11, 41, (taxi[
    set_code_reference_subscript(11, 13, 11, 36, ('passenger_count'))] >= 1))]
undo_monkey_patch()



FileNotFoundError: [Errno 2] No such file or directory: 'c:\\users\\dime\\sqlundmlinspect\\mlinspect\\example_pipelines\\taxi\\yellow_tripdata_202101_head.csv'

In [5]:
run_for_all(pipe, ['passenger_count','trip_distance'])


Original: 2.384185791015625e-07

PostgreSQL View: 6.093724250793457

PostgreSQL Materialized View: 8.352730512619019

Umbra View: 2.307494640350342

PostgreSQL CTE: 8.940905809402466

Umbra CTE: 2.25715970993042


In [6]:
run_for_all(pipe, ['passenger_count','trip_distance','PULocationID'])


Original: 0.0

PostgreSQL View: 6.783777952194214

PostgreSQL Materialized View: 9.736430168151855

Umbra View: 2.664641857147217

PostgreSQL CTE: 12.570049285888672

Umbra CTE: 2.7903082370758057


In [7]:
run_for_all(pipe, ['passenger_count','trip_distance','PULocationID','DOLocationID'])


Original: 2.384185791015625e-07

PostgreSQL View: 6.993323802947998

PostgreSQL Materialized View: 9.6921968460083

Umbra View: 2.779151678085327

PostgreSQL CTE: 15.836299419403076

Umbra CTE: 3.1126739978790283


In [8]:
run_for_all(pipe, ['passenger_count','trip_distance','PULocationID','DOLocationID','payment_type'])


Original: 2.384185791015625e-07

PostgreSQL View: 8.537209272384644

PostgreSQL Materialized View: 10.996732950210571

Umbra View: 2.854299306869507

PostgreSQL CTE: 18.561489820480347

Umbra CTE: 3.0941214561462402
